In [368]:
inputfile = 'AirQualityData.csv'

In [369]:
import pandas as pd

In [370]:
df = pd.read_csv(inputfile, encoding = "ISO-8859-1")

In [371]:
df = df.drop(['stn_code', 'sampling_date', 'agency', 'type', 'location_monitoring_station','pm2_5', 'spm'], axis = 1)

In [372]:
df.columns.values

array(['state', 'location', 'so2', 'no2', 'rspm', 'date'], dtype=object)

In [373]:
years = ['2013','2014']
df = df[df["date"].str.contains('|'.join(years), na=False)]

In [374]:
df.loc[:, 'date'] = df['date'].str.split('-', expand=True)[0]

In [375]:
df['state'] = df['state'].replace(['Uttaranchal'],'Uttarakhand')
df['state'] = df['state'].replace(['Telangana'],'Andhra Pradesh')

In [376]:
df = df.groupby(['state','location', 'date']).mean()

In [377]:
# Snippets for checking null values (set year as 2014)
# df.sort_values(by = ['state','location','date'], ascending=[True,True,False])
# df.isnull().sum()
# df[df['so2'].isnull()]
# df[df['no2'].isnull()]

In [378]:
# Only this row has NA in no2 and so2 in 2014, rspm is present for all cities in 2014
df.loc['Andhra Pradesh','Eluru','2014']['so2'] = df.loc['Andhra Pradesh','Eluru','2013']['so2']
df.loc['Andhra Pradesh','Eluru','2014']['no2'] = df.loc['Andhra Pradesh','Eluru','2013']['no2']
df.loc['Andhra Pradesh','Eluru','2014']

so2       5.459459
no2      15.189189
rspm    121.482143
Name: (Andhra Pradesh, Eluru, 2014), dtype: float64

In [379]:
df = df.reset_index(level=['state','location','date'])

In [380]:
df

,state,location,date,so2,no2,rspm
0,Andhra Pradesh,Ananthapur,2013,4.000000,12.564815,70.009259
1,Andhra Pradesh,Ananthapur,2014,4.787037,10.314815,75.990741
2,Andhra Pradesh,Chittoor,2013,4.310185,9.421296,51.268519
3,Andhra Pradesh,Chittoor,2014,5.043478,12.717391,63.944099
4,Andhra Pradesh,Eluru,2013,5.459459,15.189189,115.275000
...,...,...,...,...,...,...
404,West Bengal,Kolkata,2014,8.049180,65.786885,124.824716
405,West Bengal,Raniganj,2013,10.041667,50.750000,163.208333
406,West Bengal,Raniganj,2014,9.250000,57.980769,134.192308
407,West Bengal,South Suburban,2013,8.516471,59.565490,180.366471


In [381]:
df = df[df['date'].str.contains('2014', na=False)]
df.columns

Index(['state', 'location', 'date', 'so2', 'no2', 'rspm'], dtype='object')

In [382]:
df = df.drop(['date'], axis = 1)

In [383]:
# For normalisation if required in future
# df = (df-df.min())/(df.max()-df.min())

In [384]:
df['Mean-Pollutant-Concentration'] = df.mean(axis=1)

In [385]:
# Mean-Pollutant-Concentration
df = df.drop(['so2', 'no2', 'rspm'], axis = 1)

In [386]:
df = df.sort_values(by = ['state','Mean-Pollutant-Concentration'], ascending=[True,False])

In [387]:
df = df.groupby('state').head(5)

In [388]:
df.head(30)

,state,location,Mean-Pollutant-Concentration
31,Andhra Pradesh,Sangareddy,51.340631
5,Andhra Pradesh,Eluru,47.376931
36,Andhra Pradesh,Vijayawada,43.105870
9,Andhra Pradesh,Hyderabad,42.254452
25,Andhra Pradesh,Patancheru,41.495327
43,Arunachal Pradesh,Itanagar,27.200000
44,Arunachal Pradesh,Naharlagun,23.733333
60,Assam,Nagaon,40.130802
54,Assam,Guwahati,38.493594
66,Assam,Sivasagar,36.186603


In [389]:
df.to_csv("PollutedCities.csv", index=False)